# Create Single household load profile

Power converted to kW

In [6]:
%%capture
!pip install plotly==5.9.0

In [ ]:
#Cloning repository and changing directory
!git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
%cd RL_VPP_Thesis/data/data_training/
%ls

In [8]:
#Moving to current directory
current_folder = ''

input_folder = current_folder + 'raw_datasets/'
output_folder = current_folder + 'scenario_datasets/'

In [9]:
import numpy as np
import pandas as pd
import plotly.express as px

In [10]:
household_data = pd.read_csv(input_folder + 'consumer-00000001_glimpse_TEST_year.csv')

household_data['time'] = pd.to_datetime(household_data['time'])
household_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175201 entries, 0 to 175200
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  175201 non-null  int64         
 1   time        175201 non-null  datetime64[ns]
 2   energy      175201 non-null  float64       
 3   power       175201 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 5.3 MB


In [11]:
household_data = household_data.drop(['energy'], axis=1)
household_data.tail()

,Unnamed: 0,time,power
175196,175197,2017-12-31 23:48:00,356545
175197,175198,2017-12-31 23:51:00,360150
175198,175199,2017-12-31 23:54:00,359509
175199,175200,2017-12-31 23:57:00,428684
175200,175201,2018-01-01 00:00:00,461799


In [12]:
#Resampling time

time_sampled = household_data.resample('15min', on='time').time.sum
household_data = household_data.resample('15min', on='time').agg({'time':'min', 'power':'mean'})
household_data = household_data.drop(['time'], axis=1)
household_data = household_data.loc['2017-01-01 00:00:00':'2018-01-01 00:00:00']
household_data[household_data["power"]==""] = np.NaN
household_data.power = household_data.power.ffill()
#Convertion from mW to kW
household_data['power'] = household_data['power']/1000000
household_data.head()

,power
time,
2017-01-01 00:00:00,0.356904
2017-01-01 00:15:00,0.331488
2017-01-01 00:30:00,0.298730
2017-01-01 00:45:00,0.296286
2017-01-01 01:00:00,0.481182


In [13]:
household_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35041 entries, 2017-01-01 00:00:00 to 2018-01-01 00:00:00
Freq: 15T
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   power   35041 non-null  float64
dtypes: float64(1)
memory usage: 547.5 KB


In [14]:
#Save and plot data
#household_csv = household_data.to_csv(output_folder + 'household_load_profile.csv', index = True)
px.line(household_data["power"], title=('Single household power consuption')).update_layout(yaxis_title="kW", xaxis_rangeslider_visible=True, xaxis_range=["2017-06-01 00:00:00", "2017-07-01 00:00:00"])